In [6]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg

"""
In order for the notebook to run properly, the STO-2 input data should be in directory

    sto2_ipath      e.g.,: = './Data/level0.7/'

This directory should contain subdirectories named "xxxxx" like "03800", "03801", ....
The actual data files are in these subdirectories:
E.g. in: 
 ./03800/
   HOT03800_08190.fits
   REF03800_08191.fits
 ./03801/
   HOT03801_08192.fits
   HOT03801_08239.fits
   OTF03801_08193.fits
   OTF03801_08194.fits
   .
   .
   .
   .

Look for more comments in the code explaining what is being done.

"""


import os
import numpy as np
import glob
import sys
import datetime
from STO2_v35 import *
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord, FK5
from astropy.utils.console import ProgressBar
from pylab import *
from scipy import signal
from scipy.signal import medfilt
from scipy.interpolate import interp1d
#from loess_1d import loess_1d
from ALSFitter import *
import warnings
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
plt.rcParams['axes.formatter.useoffset'] = False
plt.rc("font", size=5)
#warnings.simplefilter('ignore', 'VerifyWarning')
warnings.filterwarnings("ignore")

from IPython.core.display import HTML
def css_styling():
    styles = open("./jupyter_custom.css", "r").read()
    return HTML(styles)
css_styling()


import time
s_today = time.strftime("%Y%m%d")
now = time.strftime("%c")
print ("Last execution: %s"  % now )



nbfile = 'STO2/EtaCar/EtaCar2_despike_OTF_3801_L0.7_2Steps.ipynb'
nbversion = '1.0g'





###############################################################################################################
# some adjustments can be made here!!!


olevel = 'level0.7vp36'




badpix = None
badpix = [479,480,481,482,483,484,485,486,487,488,489,490,491]  # forced bad pixels for all lines!
thresh = 12000.
near = 2        # mask also the next near=2 pixels on either side
npzflag = True  # flag indicating to create a numpy data file containing the despiked data
verbose = False
verbose1 = False
verbose2 = False    # get information from the mask processing
lin = 1   




# number of channels at end of spectrum replaced with value of n_rclip+1 spectral element
n_rclip = 2
# number of wing pixels corrected
nwpix = 1
# window width of median filter for determining bad pixel 
#  - if too small, detection might fail
#  - if too large, not sensitive if curvy bandpass
ww = 21
# borderpix (not used!)
borderpix = 5
rmethod = 'nan'  # can be 'nan' or 'replace'

tbad = np.array([484,485,486,487,488])
tbad = None
niirange = np.array([130,929], dtype=int)


sto2_ipath = './Data/level0.7/'     # STO-2 input files from Arizona
sto2_opath_root = './Data/'         # new cleaned files
sto2_opath = './Data/'+olevel       # new cleaned files


# directory range for Eta Carinae observation
# the directories are alternating OTF and REF/HOT observations
stdir = 3800   # directory number for starting the processing (should be REF/HOT observations)
endir = 4000   # last directory processed (should be REF/HOT observations)


###############################################################################################################
###############################################################################################################


# create output path if not exist
if not os.path.isdir(sto2_opath): os.mkdir(os.path.join(sto2_opath))  

# open a text file to log missing spectra (spectra are nan's, zeros, etc)
mlog_file = sto2_opath+'/aa_%s_missing_log.txt'%(olevel)
mlogf = open(mlog_file, 'w', encoding='utf-8')
mlogf.write(' Scan ObsID Line')

def mlogprint(ostr):
    mlogf.write(ostr+'\n')

    
mapdirs = np.arange(stdir,endir,1)
#mapdirs = np.array([3801])

if   lin==2: 
    add = '_CII_2'
    sclipl=5
    eclipl=-5
elif lin==1: 
    add = '_NII_2'
    sclipl=90
    eclipl=-140
elif lin==0: 
    add = '_NII_1'
    sclipl=90
    eclipl=-140
else: add = ''
    
print('sclip/eclip: ', sclipl, eclipl)
print()

if npzflag: print('numpy mask data file: ', sto2_opath_root+'/mask_analysis_%s_compressed.npz'%(olevel))


# get the number of files, n_files, to be processed and the number of pixels, n_pix, in a spectrum assuming
# all spectra have the same length
ffilter = '*.fits'
n_files = 0
get = 0
for j in ProgressBar(range(len(mapdirs)), ipython_widget=True):

    dirnum = mapdirs[j]
    cdirnum = '%05i'%(dirnum)
    pdirnum = '/%05i/'%(dirnum)
    
    sname = os.path.join(sto2_ipath,cdirnum,ffilter)
    afiles = sorted(glob.glob(sname))
    n_afiles = len(afiles)
    n_files += n_afiles
    if (j==get)&(n_afiles>0):
        with fits.open(afiles[0]) as hl:
            hd0 = hl[0].header
            dd0 = hl[0].data
            hd1 = hl[1].header
            dd1 = hl[1].data

            dat = dd1['DATA']
            n_pix = dat.shape[1]
    elif (j==get)&(n_afiles==0):
        get += 1


# if desired, prepare numpy file variables
if npzflag:
    idat = np.zeros((n_files,3,n_pix))
    # image of associated masks
    imsk = np.zeros((n_files,3,n_pix), dtype=np.int)
    # counter for spectra read
    ivv = np.zeros((n_files,3,n_pix))
    icd = np.zeros((n_files,3))     # for CDELT1 per line
    icv = np.zeros((n_files))       # for CRVAL1 (same for all lines)
    icp = np.zeros((n_files,3))     # for CRPIX1 per line
    cnt = 0  
    obsids = np.zeros((n_files))
    scans  = np.zeros((n_files))
    ifil  = np.ndarray((n_files), dtype=np.object)



for j in ProgressBar(range(len(mapdirs)), ipython_widget=True):

    dirnum = mapdirs[j]
    cdirnum = '%05i'%(dirnum)

    sname = os.path.join(sto2_ipath,cdirnum,'*.fits')
    odir = os.path.join(sto2_opath,cdirnum)
    afiles = sorted(glob.glob(sname))
    n_files = len(afiles)
    print('processing scan : %s with %i files'%(cdirnum, n_files))
    
    for k in range(0,n_files):
        ifile = afiles[k]
        if verbose1: print('   ', ifile)
        ofile = ifile.replace('level0.7',olevel)
        if not os.path.isdir(odir): os.mkdir(odir)
        
        with fits.open(ifile) as hl:
            hd0 = hl[0].header
            dd0 = hl[0].data
            hd1 = hl[1].header
            dd1 = hl[1].data

            dat = dd1['DATA']
            tint = np.float(hd1['OBSTIME'])
            tscan  = np.float(hd1['SCAN'])
            tobsid = np.float(hd1['OBSID'])
            ttype = hd1['TYPE']
            cols = hl[1].columns
            nrows = hl[1].data.shape[0] + 2

            nlin, npix = dat.shape

            mask = np.zeros([nlin, npix], dtype=np.int)
            # If the notebook crashes here, execute the cells below first to load the missing functions.
            mask = addHeaderBadPixels(hd1['COMMENT'], mask)
                        
            # bit 1 => set value 1 of mask:  all spectral data values are NaNs or zeros
            # bit 2 => set value 2 of mask:  major (previously identified) spikes
            # bit 3 => set value 4 to mask:  new despiking, e.g., residuals of transmission LO interference or so
            # bit 4 => set value 8 to mask:  end of spectrum outliers clipped spectral elements
            
            for i in range(nlin):
                if np.all(np.isnan(dat[i,:])): np.bitwise_or(mask[i,:], 1)   # spectra all nans
                if np.all(dat[i,:]==0.0):      np.bitwise_or(mask[i,:], 1)   # spectra all zero
                                       
                spec = np.squeeze(dat[i,:])
                smask = np.squeeze(mask[i,:])
                
                if (np.nanmin(spec) == np.nanmax(spec))|(np.all(np.isnan(spec))):
                    # this is to log files with missing data
                    ostr = '%05i %05i %4i %s'%(tscan, tobsid, i, ttype)
                    mlogprint(ostr)
                
                if lin==2:
                    ospec, omask = repSpike1D(spec/tint, smask, sclip=sclipl, eclip=eclipl, near=1, verbose=verbose2, badpix=badpix, thresh=thresh)
                else:
                    ospec, omask = repSpike1D(spec/tint, smask, sclip=sclipl, eclip=eclipl, near=1, verbose=verbose2, badpix=badpix, thresh=thresh)
                
                # we have to reverse the division by the integration time
                dat[i,:]  = ospec*tint
                mask[i,:] = omask

            dd1['DATA'] = dat

            # append the mask to the fits table (is there a better way?)
            mcol = fits.Column(name='MASK', format='1024J', array=mask)
            new_columns = hl[1].columns + mcol
            new_hdu = fits.BinTableHDU.from_columns(new_columns)


            # test if output directory exists, if not, create
            odir = os.path.join(sto2_ipath,cdirnum)
            odir = odir.replace('level0.7',olevel)
            if not os.path.isdir(odir):
                #create the output directory
                os.mkdir(os.path.join(odir))

            # save data
            fits.writeto(ofile, dd0, hd0, overwrite=True)
            hd1['history'] = '0.7v added mask with despike info; %s; user_xxx'%(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
            hd1['history'] = 'despike threshold: %f'%(thresh)
            hd1['history'] = 'adjacent pixels: %i'%(near)
            hd1['history'] = 'despiked pixel replaced with interpolated value.'
            hd1['level0']  = ('VERSION 0.7v', 'Applied on %s'%(datetime.datetime.now().strftime('%d %b %Y')))
            fits.append(ofile, new_hdu.data, hd1)

            
            # create the optional numpy array
            if npzflag:
                dd1 = new_hdu.data
                ifil[cnt] = ifile
                dat = dd1['DATA']
                idat[cnt,:,:] = dat
                mask = dd1['MASK']
                imsk[cnt,:,:] = mask
                icd[cnt,:] = dd1['CDELT1']
                icv[cnt] = hd1['CRVAL1']
                icp[cnt,:] = dd1['CRPIX1']
                ivv[cnt,lin,:] = (np.float(hd1['CRVAL1']) + (1 + np.arange(n_pix) - dd1['CRPIX1'][lin]) * dd1['CDELT1'][lin])
                tint = np.float(hd1['OBSTIME'])
                tscan  = np.float(hd1['SCAN'])
                scans[cnt] = tscan
                tobsid = np.float(hd1['OBSID'])
                obsids[cnt] = tobsid
                ttype = hd1['TYPE']
                cnt += 1


if npzflag:
                
    idat = idat[:cnt,:,:]
    imsk = imsk[:cnt,:,:]
    cnow = time.strftime("%c")


    dfile = sto2_opath_root+'/mask_analysis_%s_compressed.npz'%(olevel)
    np.savez_compressed(dfile, idat=idat, imsk=imsk, icd=icd, icv=icv, icp=icp, ivv=ivv, scans=scans, 
                        obsids=obsids, cnt=cnt, filen = ifil, olevel=olevel, nbversion=nbversion,
                        nbfile=nbfile, created=cnow, 
                        sto2_ipath=sto2_ipath)

    print('created: ', dfile)

            
            
mlogf.close()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Last execution: Tue May 19 12:55:12 2020
sclip/eclip:  90 -140

numpy mask data file:  ./Data//mask_analysis_level0.7vp36_compressed.npz


FloatProgress(value=0.0)

FloatProgress(value=0.0)

processing scan : 03800 with 2 files
processing scan : 03801 with 48 files
processing scan : 03802 with 2 files
processing scan : 03803 with 46 files
processing scan : 03804 with 2 files
processing scan : 03805 with 47 files
processing scan : 03806 with 2 files
processing scan : 03807 with 48 files
processing scan : 03808 with 2 files
processing scan : 03809 with 47 files
processing scan : 03810 with 2 files
processing scan : 03811 with 48 files
processing scan : 03812 with 2 files
processing scan : 03813 with 47 files
processing scan : 03814 with 2 files
processing scan : 03815 with 48 files
processing scan : 03816 with 2 files
processing scan : 03817 with 47 files
processing scan : 03818 with 2 files
processing scan : 03819 with 48 files
processing scan : 03820 with 2 files
processing scan : 03821 with 47 files
processing scan : 03822 with 2 files
processing scan : 03823 with 46 files
processing scan : 03824 with 2 files
processing scan : 03825 with 46 files
processing scan : 03826 w

# Cleaning procedure

as of 8/26/2019


However, the primary function file is in STO2_v35.

In [4]:
from scipy import interpolate

print('loading functions ...')

def repSpike1D(ispec1, imask1, sclip=0, eclip=0, thresh = 10000, bit2flag=False, verbose=False):
    """
    This function is still experimental, hence the many print statements for debugging.
    
    This is the latest cleaning function, based on a several step cleaning procedure.
    The first step removes and replaces the large spikes and tries to interpolate the missing
    values. There is an option for replacing the first sclip pixels with the value of pixel[sclip] 
    and the last eclip pixels with the value of pixel[eclip]. These ranges were almost impossible 
    to clean and repair for [NII], and this methods seems to be the best option since the data are 
    useless in any case. 
    The second step removes smaller spikes that have an amplitude larger than thresh compared 
    to the adjacent pixels. Here are 2 option, using the median filtered difference or the ALS difference
    of old minus filtered spectrum? Only positive spikes are removed. Only median filer used right now.
    
    Compare to subroutine setMask() below for changes in flagging:
    Meaning of mask bits (might change slightly in future by adding new values, 16 bits available):    
        bit 0 => add 1 to mask:  all spectral data values are NaNs or zeros
        bit 1 => add 2 to mask:  spikes listed in comment area of raw (level 0.5+) files.
        bit 2 => add 4 to mask:  new despiking, e.g., residuals of transmission LO interference or so
        bit 3 => add 8 to mask:  end of spectrum outliers clipped spectral elements
    
    Inputs:
        ispec1:      "dirty" spectrum
        imask1:      initial mask, most likely empty
        sclip:       number of pixels to be replaced at beginning of spectrum  (bit 3)
        eclip:       number of pixels to be replaced at end of spectrum   (bit 3)
        thresh:      threshold for identifying smaller spikes  (bit 2 set if spike)
                     (value independent of absolute pixel value; 10000 too aggressive?) 
        bit2flag:    True: interpolate the value of pixels masked in bit 2 (a.k.a., the pixels identified in the file header comment section)
                     False: ignore...
    
    Function created after test notebook: notebooks/STO2/EtaCar/EtaCar2_spike_cleaning_single_spectrum_v4_testing.ipynb
        
    initially created: 8/21/2019, VTO
    """
    
    ### Step 1
    
    # identify the large spikes and set mask
    # set bit 3 for "end of spectrum outliers"
    dtt = type(ispec1)
    if type(ispec1)==type(np.zeros(2)):
        spec1 = ispec1.copy()
    else:
        spec1 = ispec1.value
    mask1 = imask1.copy()
    if verbose: print(spec1.shape)
    
    # Step 1
    # clip ends of scan first in case there are major spikes.
    # set bit 3
    if sclip>0:
        #sclip = 70
        # replace the beginning sclip pixels with the value of the next inside pixel
        if spec1[sclip] < 0.5E9:
            spec1[:sclip] = np.ones(sclip) * spec1[sclip]
            # set the mask
            mask1[:sclip] = np.bitwise_or(mask1[:sclip], 8)   
        else:
            print('Error. repSpike1D: spectrum in pixel sclip is spike.')
        
    if np.abs(eclip)>0:
        #eclip = -100
        
        # replace the eclip pixels at end of scan with the value of the pixel at eclip-1
        if spec1[eclip] < 0.5E9:
            spec1[eclip:] = np.ones(np.abs(eclip)) * spec1[eclip]
            # set the mask
            mask1[eclip:] = np.bitwise_or(mask1[eclip:], 8)   
        else:
            print('Error. repSpike1D: spectrum in pixel eclip is spike.')
            
    # Step 2
    # Identify and mask major spikes with counts over 5E8, set to nan.
    # set bit 2 for "new de-spiking"
    args = np.argwhere(spec1 > 0.5e9)
    for arg in args:
        spec1[arg] = np.nan
        mask1[arg] = np.bitwise_or(mask1[arg], 4)
        if arg<spec1.size-1:    
            spec1[arg+1] = np.nan
            mask1[arg+1] = np.bitwise_or(mask1[arg+1], 4)
            if arg<spec1.size-2:    
                spec1[arg+2] = np.nan
                mask1[arg+2] = np.bitwise_or(mask1[arg+2], 4)
        if arg>1:
            spec1[arg-1] = np.nan
            mask1[arg-1] = np.bitwise_or(mask1[arg-1], 4)
            if arg>2:
                spec1[arg-2] = np.nan
                mask1[arg-2] = np.bitwise_or(mask1[arg-2], 4)
    idx = np.arange(spec1.size)
    
    try:
        # get the nan groups:
        midx = idx[np.where(mask1==4)]
        gmidx = list(groupSequence(midx))

        seps = [gmidx[0][0]]
        for i in range(len(gmidx)-1): seps.append(gmidx[i+1][0] - gmidx[i][-1])
        seps.append(spec1.size - gmidx[-1][-1])
    except:
        pass
    
    #if verbose: print('Separations of groups: ', seps)
    
    # create an index array for the spectrum (e.g., replacement for velocity array)
    pidx  = np.arange(spec1.shape[0])
    pidx1 = np.arange(spec1.shape[0])
    
    # group all pixels with mask flag 4 (or flag2 and flag 4 but not flag 8)
    mpidx1 = pidx[np.where(np.bitwise_and(mask1, 4))]
    if bit2flag:
        mpidx1 = pidx[(np.bitwise_and(mask1, 2)|np.bitwise_and(mask1, 4))&(np.bitwise_and(mask1, 8))]
    if mpidx1.size>0:
        gmidx1 = list(groupSequence(mpidx1))
        n_gmidx1 = len(gmidx1)
        
        seps1 = [gmidx1[0][0]]
        for i in range(len(gmidx1)-1): seps1.append(gmidx1[i+1][0] - gmidx1[i][-1])
        seps1.append(spec1.size - gmidx1[-1][-1])
    else:
        n_gmidx1 = 0
        
    # interpolation parameters
    s=0.0   # smoothing condition, if 0.0, do interpolation if no weights provided
    k=3     # order of spline fit -> use cubic splines
    
    for i in range(n_gmidx1):
        g0 = gmidx1[i]   # get i-th group
        ps = g0[0]       # first element in group
        pe = g0[-1]      # last element in group
        ds = 30          # repair interval on start side
        if ds > seps1[i]: ds = seps1[i]
        de = 30          # repair interval on end side
        if de > seps1[i+1]: de = seps1[i+1]
        ds = int(ds)
        de = int(de)
        
        # the arrays below exclude ps to pe, the range we want to interpolate
        isp = np.hstack((spec1[ps-ds:ps],spec1[pe+1:pe+de]))
        ipi = np.hstack(( pidx1[ps-ds:ps], pidx1[pe+1:pe+de]))
        
        # interpolate missing/masked values
        if isp.size>3:
            it = interpolate.splrep(ipi, isp, k=k)
            spec1arep = interpolate.splev(pidx[ps:pe+1], it, der=0)
            spec1[ps:pe+1] = spec1arep
        else:
            print('ERROR: no interpolation of major spikes....')
            print(ipi.shape, isp.shape, g0)

    ##### Step 3
    # Remove additional smaller spikes
    # set bit 2 for "new de-spiking"
    
    # median filter option
    ks = 7         # median filter kernel
    out2 = medfilt(spec1, kernel_size=ks)
    
    # ALS option
    # out3 = als(spec1, lam=0.1, p=0.1)

    #thresh = 60000    # spike threshold, spikes below threshold can be removed later.
    tsel = np.argwhere((spec1 - out2)>thresh)
    # plt.plot(idx[tsel],spec1[tsel],'*')
    # mask the small spikes with 4
    mask1[tsel] = np.bitwise_or(mask1[tsel], 4)
    # remove pixel before and after since we do not know if they are affected.
    tsel1 = tsel[np.where((tsel>1)&(tsel<1023))]
    mask1[tsel1-1] = np.bitwise_or(mask1[tsel1-1], 4)
    mask1[tsel1+1] = np.bitwise_or(mask1[tsel1+1], 4)
    if verbose: print('small spikes (bit 2, flag value 4) at (+/-1): ', np.squeeze(tsel))
    
    ##################
    # now, try to interpolate the identified pixels
    
    mpidx1 = pidx[np.argwhere(np.bitwise_and(mask1, 4))]
    n_mpidx1 = len(mpidx1)
    
    s=0.0
    k=3
    
    for i in range(n_mpidx1):
        e0 = int(mpidx1[i])   # get i-th element
        #print(i, e0)
        ds = int(15)
        if ds > e0: ds = e0
        de = int(15)
        if de > spec1.size: de = int(spec1.size - e0 - 1)
        
        # the arrays below exclude ps to pe, the range we want to interpolate
        e0 = int(e0)
        isp = np.squeeze(spec1[e0-ds:e0+de])
        ipi =  np.squeeze(pidx1[e0-ds:e0+de])
        ims = np.squeeze(mask1[e0-ds:e0+de])
        esel = np.argwhere(np.bitwise_and(ims, 4)==0)
        esel2 = np.squeeze(np.argwhere(np.bitwise_and(ims, 4)))
        ispr = np.squeeze(isp[esel])
        ipir = np.squeeze(ipi[esel])
        imsr = np.squeeze(ims[esel])
        #print(e0, ds, esel2, e0-ds+esel2)
        
        try:
            it = interpolate.splrep(ipir, ispr, k=k)
            sprep = np.squeeze(interpolate.splev(ipi[esel2], it, der=0))
            #print('   org: ', isp[esel2])
            #print(isp)
            isp[esel2] = sprep
            #print(isp)
            #print('   rep: ', spec1a[e0-ds+esel2])
            spec1[e0-ds+esel2] = sprep
        except:
            print(ispr.size)

    return spec1, mask1




from itertools import groupby, cycle 
  
def groupSequence(l): 
    temp_list = cycle(l) 
    
    next(temp_list) 
    groups = groupby(l, key = lambda j: j + 1 == next(temp_list)) 
    for k, v in groups: 
        if k: 
            yield tuple(v) + (next((next(groups)[1])), ) 



def addHeaderBadPixels(icom, imask):
    """
    Add the bad pixels listed in the header in the comment section as determined by scooper, the Level 0 to Level 0.5 program.
    The flag value for bad pixels here is 2.
    
    However, the header information should have been added when creating the mask! If it is added again, it should not change 
    the already existing information in case the functions has been invoked more than once.
    """
    nmask = imask.copy()
    try:
        #icom = hd1['COMMENT']
        com = np.genfromtxt(icom, dtype=None, names=['brd', 'inp', 'chan', 'oldv', 'newv', 'line', 'pixel'],
                            delimiter=[3,3,6,12,13,5,5], skip_header=2)

        # bit 1 => add 2 to mask: originally despiked in scooper
        # first NII line, second NII line, and [OI] line
        nmask[0,com['chan'][np.where((com['brd']==1)&(com['inp']==1))]-1] = np.bitwise_or(nmask[0,com['chan'][np.where((com['brd']==1)&(com['inp']==1))]-1], 2)
        nmask[1,com['chan'][np.where((com['brd']==1)&(com['inp']==2))]-1] = np.bitwise_or(nmask[1,com['chan'][np.where((com['brd']==1)&(com['inp']==2))]-1], 2)
        nmask[2,com['chan'][np.where((com['brd']==3)&(com['inp']==1))]-1] = np.bitwise_or(nmask[2,com['chan'][np.where((com['brd']==3)&(com['inp']==1))]-1], 2)
        if verbose: print('2: ', com['chan'][np.where((com['brd']==3)&(com['inp']==1))]-1)
    except:
        pass
    return nmask

print('Done.')


loading functions ...
Done.
